In [ ]:
import os
import sys
current_dir = os.getcwd()
target_dir_name = 'ba_code_project'
while True:
    # Check if the target directory exists in the current directory
    potential_target = os.path.join(current_dir, target_dir_name)
    if os.path.isdir(potential_target):
        code_root_dir = potential_target
        break
    # Move one level up
    parent_dir = os.path.dirname(current_dir)
    # If we're at the root of the file system and still haven't found it, stop
    if parent_dir == current_dir:
        code_root_dir = None
        break
    current_dir = parent_dir
if code_root_dir:
    # Add the found target directory to sys.path
    sys.path.append(code_root_dir)
else:
    print(f'Target directory not found.')

In [ ]:
import numpy as np
import pandas as pd
import ast
from src.features.get_first_and_last_x_y_coordinates import get_first_and_last_x_y_coordinates
from src.features.get_x_y_tuple_list import get_x_y_tuple_list
from src.visualization.plot_vehicle_tracks_in_notebook import plot_vehicle_tracks_in_notebook

%matplotlib inline

In [ ]:
# to access while testing
# intersection_name = 'k729_2022'
# intersection_name = 'k733_2018'
intersection_name = 'k733_2020'
data_path = f'{code_root_dir}/data/processed/{intersection_name}_cuid.csv'

df_cuid = pd.read_csv(data_path)
df_cuid_grouped_path = data_path.replace('.csv', '_grouped.csv')
df_cuid_grouped = pd.read_csv(df_cuid_grouped_path)
df_cuid_grouped['x'] = df_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
df_cuid_grouped['y'] = df_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))
list_x_y_tuples = get_x_y_tuple_list(df_cuid_grouped, ['x','y'])
first_last_x_coords, first_last_y_coords = get_first_and_last_x_y_coordinates(list_x_y_tuples)
X_intersection = np.array([first_last_x_coords, first_last_y_coords]).T

In [ ]:
print(len(df_cuid_grouped['track_id']))
print(intersection_name)

# plot simple dtw alignment + cost matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# =========================
# Step 1: Define Two Unique Sample Sequences
# =========================

# Define unique sequences for clarity
sequence_a = np.array([2, 5, 8, 11])
sequence_b = np.array([1, 3, 5, 7, 9, 10, 12])

# =========================
# Step 2: Compute the Cost Matrix
# =========================

cost_matrix = cdist(sequence_a[:, np.newaxis], sequence_b[:, np.newaxis], metric='euclidean')

# =========================
# Step 3: Compute the Accumulated Cost Matrix (DTW Matrix)
# =========================

accumulated_cost = np.full((len(sequence_a), len(sequence_b)), np.inf)
accumulated_cost[0, 0] = cost_matrix[0, 0]

for j in range(1, len(sequence_b)):
    accumulated_cost[0, j] = cost_matrix[0, j] + accumulated_cost[0, j-1]
for i in range(1, len(sequence_a)):
    accumulated_cost[i, 0] = cost_matrix[i, 0] + accumulated_cost[i-1, 0]
for i in range(1, len(sequence_a)):
    for j in range(1, len(sequence_b)):
        accumulated_cost[i, j] = cost_matrix[i, j] + min(
            accumulated_cost[i-1, j-1],  # Diagonal (Match)
            accumulated_cost[i-1, j],    # Left (Insertion)
            accumulated_cost[i, j-1]     # Up (Deletion)
        )

# =========================
# Step 4: Compute the Optimal Warping Path
# =========================

i, j = len(sequence_a) - 1, len(sequence_b) - 1
path = [(i, j)]
while i > 0 or j > 0:
    if i == 0:
        j -= 1
    elif j == 0:
        i -= 1
    else:
        choices = [
            accumulated_cost[i-1, j-1],
            accumulated_cost[i-1, j],
            accumulated_cost[i, j-1]
        ]
        move = np.argmin(choices)
        if move == 0:
            i -= 1
            j -= 1
        elif move == 1:
            i -= 1
        else:
            j -= 1
    path.append((i, j))

path = np.array(path[::-1])

# =========================
# Plot 1: Sequences Alignment with DTW Path
# =========================

fig1, ax1 = plt.subplots(figsize=(10, 8))
ax1.plot(range(len(sequence_a)), sequence_a, marker='o', label='Sequence A', color='blue', markersize=8, linewidth=2)
ax1.plot(range(len(sequence_b)), sequence_b, marker='s', label='Sequence B', color='green', markersize=8, linewidth=2)

for (i, j) in path:
    ax1.plot([i, j], [sequence_a[i], sequence_b[j]], 'r--', linewidth=1.5, alpha=0.5)

ax1.set_xlabel('Index', fontsize=16)
ax1.set_ylabel('Value', fontsize=16)
ax1.legend(fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='both', which='major', labelsize=12)

plt.tight_layout()
plt.show()

# =========================
# Plot 2: DTW Accumulated Cost Matrix with Optimal Warping Path
# =========================

fig2, ax2 = plt.subplots(figsize=(10, 8))
cax = ax2.imshow(accumulated_cost, origin='lower', cmap='Blues', interpolation='nearest', aspect='auto',
                 extent=[-0.5, len(sequence_b) - 0.5, -0.5, len(sequence_a) - 0.5])
cbar = fig2.colorbar(cax, ax=ax2, label='Accumulated Cost', fraction=0.046, pad=0.04)
cbar.set_label('Accumulated Cost', fontsize=16)

ax2.plot(path[:, 1], path[:, 0], 'r', linewidth=2, label='Optimal Warping Path')

# Annotate each cell with its cost value
for i in range(len(sequence_a)):
    for j in range(len(sequence_b)):
        ax2.text(j, i, f'{accumulated_cost[i, j]:.1f}', ha='center', va='center', color='black', fontsize=12)

ax2.set_xticks(np.arange(len(sequence_b)))
ax2.set_yticks(np.arange(len(sequence_a)))
ax2.set_xlim(-0.5, len(sequence_b) - 0.5)
ax2.set_ylim(-0.5, len(sequence_a) - 0.5)

ax2.set_xlabel('Sequence B Index', fontsize=16)
ax2.set_ylabel('Sequence A Index', fontsize=16)
ax2.legend(fontsize=14)

ax2.grid(False)

plt.tight_layout()
plt.show()


# plot dtw alignment path + cost matrix intersection k729_2022

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from src.visualization.plot_vehicle_tracks_in_notebook import plot_vehicle_tracks_in_notebook

# =========================
# Step 1: Define Your Sequences
# =========================

track_id_a = 48
track_id_b = 7
x_a = np.array(df_cuid_grouped['x'][track_id_a])
y_a = np.array(df_cuid_grouped['y'][track_id_a])
x_b = np.array(df_cuid_grouped['x'][track_id_b])
y_b = np.array(df_cuid_grouped['y'][track_id_b])

# =========================
# Step 2: Compute the Cost Matrix
# =========================

sequence_a_points = np.column_stack((x_a, y_a))
sequence_b_points = np.column_stack((x_b, y_b))
cost_matrix = cdist(sequence_a_points, sequence_b_points, metric='euclidean')

# =========================
# Step 3: Compute the Accumulated Cost Matrix (DTW Matrix)
# =========================

len_a, len_b = len(sequence_a_points), len(sequence_b_points)
accumulated_cost = np.full((len_a, len_b), np.inf)
accumulated_cost[0, 0] = cost_matrix[0, 0]

for j in range(1, len_b):
    accumulated_cost[0, j] = cost_matrix[0, j] + accumulated_cost[0, j-1]
for i in range(1, len_a):
    accumulated_cost[i, 0] = cost_matrix[i, 0] + accumulated_cost[i-1, 0]
for i in range(1, len_a):
    for j in range(1, len_b):
        accumulated_cost[i, j] = cost_matrix[i, j] + min(
            accumulated_cost[i-1, j-1], 
            accumulated_cost[i-1, j],    
            accumulated_cost[i, j-1]     
        )

# =========================
# Step 4: Compute the Optimal Warping Path
# =========================

i, j = len_a - 1, len_b - 1
path = [(i, j)]
while i > 0 or j > 0:
    if i == 0:
        j -= 1
    elif j == 0:
        i -= 1
    else:
        choices = [
            accumulated_cost[i-1, j-1],
            accumulated_cost[i-1, j],
            accumulated_cost[i, j-1]
        ]
        move = np.argmin(choices)
        if move == 0:
            i -= 1
            j -= 1
        elif move == 1:
            i -= 1
        else:
            j -= 1
    path.append((i, j))
path = np.array(path[::-1])

# =========================
# Plot 1: Sequences Alignment with DTW Path
# =========================

fig1, ax1 = plt.subplots(figsize=(10, 10))
plot_vehicle_tracks_in_notebook(ax1, df_cuid, df_cuid_grouped, color='gray', alpha=0.1)
ax1.plot(x_a, y_a, marker='o', label=f'Sequence track_id {track_id_a}', color='blue')
ax1.plot(x_b, y_b, marker='s', label=f'Sequence track_id {track_id_b}', color='green')

for (i, j) in path:
    ax1.plot([x_a[i], x_b[j]], [y_a[i], y_b[j]], 'r--', linewidth=1.5)

# ax1.set_xlabel('X', fontsize=16)
# ax1.set_ylabel('Y', fontsize=16)
# ax1.legend(fontsize=14)
# set xtick fontsize
ax1.tick_params(axis='x', labelsize=16)
# set ytick fontsize
ax1.tick_params(axis='y', labelsize=16)
ax1.grid(True, alpha=0.3)
# ax1.tick_params(axis='both', which='major', labelsize=12)

# =========================
# Plot 2: DTW Accumulated Cost Matrix with Optimal Warping Path
# =========================

fig2, ax2 = plt.subplots(figsize=(10, 10))
cax = ax2.imshow(accumulated_cost, origin='lower', cmap='Blues', interpolation='nearest', aspect='auto',
                 extent=[-0.5, len_b - 0.5, -0.5, len_a - 0.5])
cbar = fig2.colorbar(cax, ax=ax2, orientation='vertical', fraction=0.046, pad=0.04)
cbar.set_label('Accumulated Cost', fontsize=14)

ax2.plot(path[:,1], path[:,0], 'r', linewidth=2, label='Optimal Warping Path')
ax2.set_xlabel(f'Sequence track_id {track_id_b} index', fontsize=16)
ax2.set_ylabel(f'Sequence track_id {track_id_a} index', fontsize=16)
ax2.legend(fontsize=14)

ax2.set_xticks(np.arange(0, len_b, 5))
ax2.set_yticks(np.arange(0, len_a, 5))
ax2.set_xlim(-0.5, len_b - 0.5)
ax2.set_ylim(-0.5, len_a - 0.5)
ax2.tick_params(axis='both', which='major', labelsize=12)

plt.show()


# plot optics + reachability plot k729_2022

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import OPTICS

# =========================
# Step 1: Define Your Data
# =========================

# Example data: list of lists containing (x, y) tuples
# Replace this with your actual data
data = list_x_y_tuples

# =========================
# Step 2: Flatten the Data
# =========================

# Flatten the list of lists into a single list of (x, y) tuples
flattened_data = [point for sequence in data for point in sequence]

# Convert the list of tuples into a NumPy array
X = np.array(flattened_data)  # Shape: (n_points, 2)

# =========================
# Step 3: Apply OPTICS Clustering
# =========================

# Initialize OPTICS with desired parameters
# Adjust min_samples and min_cluster_size based on your data's characteristics
optics_model = OPTICS(min_samples=7,
                      cluster_method='dbscan',
                      metric='manhattan',
                      max_eps=4.9)

# Fit the OPTICS model to the data
optics_model.fit(X_intersection)

# Extract reachability distances and the ordering of points
reachability = optics_model.reachability_
ordering = optics_model.ordering_
clusters = optics_model.labels_

# =========================
# Step 4: Handle NaN or Inf in Reachability
# =========================

# Check for NaN or Inf in reachability distances
if np.any(np.isnan(reachability)):
    print("Warning: Reachability distances contain NaN values. Replacing them with the maximum finite reachability distance.")
    # Replace NaN with the maximum finite reachability distance
    max_finite_reach = np.max(reachability[np.isfinite(reachability)])
    reachability = np.where(np.isnan(reachability), max_finite_reach + 1, reachability)

if np.any(np.isinf(reachability)):
    print("Warning: Reachability distances contain Inf values. Replacing them with the maximum finite reachability distance.")
    # Replace Inf with a value slightly larger than the maximum finite reachability distance
    max_finite_reach = np.max(reachability[np.isfinite(reachability)])
    reachability = np.where(np.isinf(reachability), max_finite_reach + 1, reachability)

# =========================
# Step 5: Plot Reachability Plot and Clustering Results
# =========================

# Create a figure with two vertically stacked subplots
fig, ax1 = plt.subplots(1, 1, figsize=(10, 10))  # Adjust figsize as needed
fig, ax2 = plt.subplots(1, 1, figsize=(10, 10))  # Adjust figsize as needed

# -------------------------
# Top Subplot: Reachability Plot
# -------------------------

# Define unique labels (clusters)
unique_labels = np.unique(clusters)
n_clusters = len(unique_labels) - (1 if -1 in clusters else 0)  # Exclude noise

# Generate colors for clusters, excluding noise
colors = plt.cm.viridis(np.linspace(0, 1, n_clusters))

# Create a color mapping for clusters
color_dict = {label: color for label, color in zip(unique_labels[unique_labels != -1], colors)}
color_dict[-1] = 'gray'  # Assign gray color for noise

# Sort reachability distances according to the ordering
sorted_reachability = reachability[ordering]
sorted_labels = clusters[ordering]
sorted_colors = [color_dict[label] for label in sorted_labels]
space = np.arange(len(X_intersection))

# Plot the reachability plot with individual colors per segment
for i in range(len(space) - 1):
    ax1.plot(space[i:i+2], sorted_reachability[i:i+2], marker='.', linestyle='-', color=sorted_colors[i])

# Fill between the reachability distances with individual colors per segment
for i in range(len(space) - 1):
    ax1.fill_between(space[i:i+2], sorted_reachability[i:i+2], np.min(sorted_reachability), alpha=0.3, color=sorted_colors[i])


# Set plot limits based on reachability data
ax1.set_ylim([0, np.max(sorted_reachability) * 1.1])
ax1.set_xlabel('Ordered Points', fontsize=16)
ax1.set_ylabel('Reachability Distance', fontsize=16)
ax1.grid(True, alpha=0.1)

# -------------------------
# Bottom Subplot: Cluster Extraction Plot
# -------------------------

# Assign black color for noise if present
if -1 in clusters:
    color_dict[-1] = [0, 0, 0, 1]  # Black color for noise

# Plot each cluster
for label in unique_labels:
    class_member_mask = (clusters == label)
    xy = X_intersection[class_member_mask]
    
    if label == -1:
        # Plot noise
        ax2.plot(xy[:, 0], xy[:, 1], 'o',
                 color='gray',
                 markersize=6,
                 label='Noise',
                 zorder=100)
    else:
        # Plot clustered points
        ax2.plot(xy[:, 0], xy[:, 1], 'o',
                 color=tuple(color_dict[label]),
                 markersize=6,
                 label=f'Cluster {label}',
                 zorder=100)

# Set plot limits based on data
all_x = X[:, 0]
all_y = X[:, 1]
ax2.set_xlim([np.min(all_x) - 1, np.max(all_x) + 1])
ax2.set_ylim([np.min(all_y) - 1, np.max(all_y) + 1])
plot_vehicle_tracks_in_notebook(ax2, df_cuid, df_cuid_grouped, color='gray', alpha=0.1)

# Set labels and title
ax2.set_xlabel('X', fontsize=16)
ax2.set_ylabel('Y', fontsize=16)
ax2.legend(loc='best', fontsize=16)
ax2.grid(True, alpha=0.1)


# =========================
# Final Adjustments and Show Plot
# =========================

plt.tight_layout()
plt.show()


# plot vehicle paths k729_2022

In [ ]:
%matplotlib inline
track_id = 69
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Plot background vehicle tracks with customized appearance
plot_vehicle_tracks_in_notebook(ax, df_cuid, df_cuid_grouped, color='gray', alpha=0.3)
ax.grid(True, alpha=0.3)

# Highlight the specific vehicle track with a larger linewidth and red color
ax.plot(df_cuid_grouped['x'][track_id], df_cuid_grouped['y'][track_id], 
        color='red', linestyle='-', linewidth=3, label=f'vehicle_id {track_id}')

# Adjust label sizes, legend, and tick parameters
ax.set_xlabel('X', fontsize=16)
ax.set_ylabel('Y', fontsize=16)
ax.legend(fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=12)

# Repeat for another track ID
track_id = 4
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

plot_vehicle_tracks_in_notebook(ax, df_cuid, df_cuid_grouped, color='gray', alpha=0.3)
ax.grid(True, alpha=0.3)

ax.plot(df_cuid_grouped['x'][track_id], df_cuid_grouped['y'][track_id], 
        color='red', linestyle='-', linewidth=3, label=f'vehicle_id {track_id}')

ax.set_xlabel('X', fontsize=16)
ax.set_ylabel('Y', fontsize=16)
ax.legend(fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=12)

plt.show()


# plot vehicle paths k733_2018

In [ ]:
import matplotlib.pyplot as plt

# Set track_id for demonstration and display length of x-coordinates for verification
track_id = 43
print("Number of points:", len(df_cuid_grouped['x'][track_id]))

# Define colors for plotting
colors = ['blue', 'green', 'red', 'purple', 'orange', 'brown', 'pink', 'gray', 'olive', 'cyan']

# Set up the figure and axis with increased figure size
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Highlight specific points in the track as potential outliers with larger, distinct markers
for i, (x, y) in enumerate(zip(df_cuid_grouped['x'][track_id], df_cuid_grouped['y'][track_id])):
    if i in range(121, 143) and i % 2 == 0:  # Highlight every other point in the range
        label = ''
        if i == 142:
            label = f'Point outliers for vehicle_id {track_id}'
        ax.scatter(x, y, color='red', s=50, label=label, zorder=1000, edgecolors='white')  # Larger size for emphasis

# Plot the main vehicle track with a distinct color
ax.plot(df_cuid_grouped['x'][track_id], df_cuid_grouped['y'][track_id], 
        linestyle='-', color='blue', alpha=0.6, linewidth=2, zorder=100)

# Background plot of other tracks for context
plot_vehicle_tracks_in_notebook(ax, df_cuid, df_cuid_grouped, color='gray', alpha=0.3)

# Set grid transparency, labels, legend, and tick parameters for improved readability
ax.grid(True, alpha=0.3)
ax.set_xlabel('X', fontsize=16)
ax.set_ylabel('Y', fontsize=16)
ax.legend(fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=12)

plt.show()


#### plot with zoomed in window

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import matplotlib.patches as patches

# Set track_id for demonstration and display length of x-coordinates for verification
track_id = 43
print("Number of points:", len(df_cuid_grouped['x'][track_id]))

# Define colors for plotting
colors = ['blue', 'green', 'red', 'purple', 'orange', 'brown', 'pink', 'gray', 'olive', 'cyan']

# Set up the figure and axis with increased figure size
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# Highlight specific points in the track as potential outliers with larger, distinct markers
for i, (x, y) in enumerate(zip(df_cuid_grouped['x'][track_id], df_cuid_grouped['y'][track_id])):
    if i in range(121, 143) and i % 2 == 0:  # Highlight every other point in the range
        label = ''
        if i == 142:
            label = f'Point outliers for vehicle_id {track_id}'
        ax.scatter(x, y, color='red', s=50, label=label, zorder=1000, edgecolors='white')  # Larger size for emphasis

# Plot the main vehicle track with a distinct color
ax.plot(df_cuid_grouped['x'][track_id], df_cuid_grouped['y'][track_id], 
        linestyle='-', color='blue', alpha=0.6, linewidth=2, zorder=100)

# Background plot of other tracks for context
plot_vehicle_tracks_in_notebook(ax, df_cuid, df_cuid_grouped, color='gray', alpha=0.3)

# Set grid transparency, labels, legend, and tick parameters for improved readability
ax.grid(True, alpha=0.3)
ax.set_xlabel('X', fontsize=16)
ax.set_ylabel('Y', fontsize=16)
ax.legend(fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=12)

# Define the zoom area coordinates
x_min = min(df_cuid_grouped['x'][track_id][121:143]) - 5
x_max = max(df_cuid_grouped['x'][track_id][121:143]) + 5
y_min = min(df_cuid_grouped['y'][track_id][121:143]) - 5
y_max = max(df_cuid_grouped['y'][track_id][121:143]) + 5

# Create inset for zoomed-in area with a red border, placing it toward the bottom left
inset_ax = inset_axes(ax, width="50%", height="50%", loc="lower left", borderpad=2)
inset_ax.plot(df_cuid_grouped['x'][track_id], df_cuid_grouped['y'][track_id], linestyle='-', color='blue', alpha=0.6, linewidth=2)

# Plot the background tracks in the zoomed inset as well
plot_vehicle_tracks_in_notebook(inset_ax, df_cuid, df_cuid_grouped, color='gray', alpha=0.3)

# Highlight points in the zoomed inset
inset_ax.scatter([df_cuid_grouped['x'][track_id][i] for i in range(121, 143) if i % 2 == 0],
                 [df_cuid_grouped['y'][track_id][i] for i in range(121, 143) if i % 2 == 0],
                 color='red', s=50, zorder=1000, edgecolors='white')

# Set limits for zoomed inset and add a red border
inset_ax.set_xlim(x_min, x_max)
inset_ax.set_ylim(y_min, y_max)
inset_ax.set_xticks([])
inset_ax.set_yticks([])
for spine in inset_ax.spines.values():
    spine.set_edgecolor('red')
    spine.set_linewidth(2)

# Connect inset to main plot with red lines
mark_inset(ax, inset_ax, loc1=2, loc2=4, fc="none", ec="red")

# Add semi-transparent overlays to dim areas outside the zoomed-in window
alpha_dim = 0.2  # Set lower alpha for dimming

# Bottom dim
ax.add_patch(patches.Rectangle((ax.get_xlim()[0], ax.get_ylim()[0]), ax.get_xlim()[1] - ax.get_xlim()[0], y_min - ax.get_ylim()[0], 
                               color='white', alpha=alpha_dim, zorder=2000))
# Top dim
ax.add_patch(patches.Rectangle((ax.get_xlim()[0], y_max), ax.get_xlim()[1] - ax.get_xlim()[0], ax.get_ylim()[1] - y_max, 
                               color='white', alpha=alpha_dim, zorder=2000))
# Left dim
ax.add_patch(patches.Rectangle((ax.get_xlim()[0], y_min), x_min - ax.get_xlim()[0], y_max - y_min, 
                               color='white', alpha=alpha_dim, zorder=2000))
# Right dim
ax.add_patch(patches.Rectangle((x_max, y_min), ax.get_xlim()[1] - x_max, y_max - y_min, 
                               color='white', alpha=alpha_dim, zorder=2000))

plt.show()


# plot itakura parallelogram

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyts.metrics import itakura_parallelogram
from pyts.metrics.dtw import _get_itakura_slopes
from tslearn.metrics import dtw_path_from_metric

def plot_itakura(n_timestamps_1, n_timestamps_2, max_slope=1.):
    """Plot Itakura parallelogram with a light gray background and light blue highlighted region."""
    region = itakura_parallelogram(n_timestamps_1, n_timestamps_2, max_slope)
    max_slope_og = max_slope
    max_slope, min_slope = _get_itakura_slopes(
        n_timestamps_1, n_timestamps_2, max_slope)
    
    # Initialize plot
    fig, ax = plt.subplots(figsize=(10, 8))
    mask = np.zeros((n_timestamps_2, n_timestamps_1))
    
    # Fill in parallelogram region with light blue color
    for i, (j, k) in enumerate(region.T):
        mask[j:k, i] = 1.
    ax.imshow(mask, origin='lower', cmap='Blues', alpha=0.3)
    ax.set_facecolor('#f7f7f7')  # Light gray background

    # Set axis limits and grid
    sz = max(n_timestamps_1, n_timestamps_2)
    x = np.arange(-1, sz + 1)

    # Define lines with modern colors
    low_max_line = ((n_timestamps_2 - 1) - max_slope * (n_timestamps_1 - 1)) + max_slope * x
    up_min_line = ((n_timestamps_2 - 1) - min_slope * (n_timestamps_1 - 1)) + min_slope * x
    diag = (n_timestamps_2 - 1) / (n_timestamps_1 - 1) * x

    ax.plot(x, diag, color='black', lw=1, label="Diagonal")
    ax.plot(x, max_slope * x, color='#1f77b4', lw=1.5, label="$Lower_1$")
    ax.plot(x, up_min_line, color='#d62728', lw=1.5, label="$Lower_2$")
    ax.plot(x, min_slope * x, color='#ff7f0e', lw=1.5, label="$Upper_1$")
    ax.plot(x, low_max_line, color='#2ca02c', lw=1.5, label="$Upper_2$")

    # Customize grid and axis settings
    ax.set_xticks(np.arange(-.5, n_timestamps_1, 5), minor=True)
    ax.set_yticks(np.arange(-.5, n_timestamps_2, 5), minor=True)
    ax.grid(which='minor', color='#bfbfbf', linestyle='--', linewidth=0.5)
    ax.set_xlim((-0.5, n_timestamps_1 - 0.5))
    ax.set_ylim((-0.5, n_timestamps_2 - 0.5))

    # Smaller legend
    ax.legend(loc='upper left', fontsize='large')
    plt.show()

# Generate separate plots for each slope value for the (50, 25) matrix
slopes = [1.5, 4.5]
for slope in slopes:
    plot_itakura(50, 25, max_slope=slope)


# create plot of feature list lengths

In [ ]:
# Calculate the length of each list in df_cuid_grouped['x']
lengths = df_cuid_grouped['x'].apply(len)

# Truncate lengths at 200 by capping values greater than 200
cutoff = 200
lengths_truncated = lengths.apply(lambda x: x if x <= cutoff else cutoff)

# Plotting the histogram with the x-axis starting at 0
plt.figure(figsize=(10, 6))
plt.hist(lengths_truncated, color='b', bins=50, edgecolor='white')
plt.xlabel("Time Series Length", fontsize=16)
plt.ylabel("Frequency", fontsize=16)

# Set x-ticks, starting from 0, and label the last one as ">cutoff"
xticks = np.arange(0, cutoff + 10, 20)  # Custom ticks from 0 to cutoff
xticks = np.append(xticks, cutoff)  # Ensure the last tick is cutoff
plt.xticks(xticks, labels=[f">{cutoff}" if x == cutoff else str(int(x)) for x in xticks], fontsize=16)
plt.yticks(fontsize=16)

plt.xlim(left=0)  # Start x-axis from 0
plt.grid(True, alpha=0.1)
plt.show()

# plot dtw distance matrix

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

dtw_distance_matrix_path = f'{code_root_dir}/data/processed/{intersection_name}_diff_itakura_slope_dtw_matrices.json'

# Load DTW distance matrix data
with open(dtw_distance_matrix_path) as f:
    dtw_distance_matrix = json.load(f)

# Choose a random key from the DTW matrix keys
dtw_keys = list(dtw_distance_matrix.keys())
# dtw_key = np.random.choice(dtw_keys, 1)[0]
dtw_key = 'itakura_2.60_dist_euclidean'


# Set font size globally
plt.rcParams.update({'font.size': 16})

# Plot the DTW distance matrix
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

dtw_dist_matrix = np.array(dtw_distance_matrix[dtw_key])
np.fill_diagonal(dtw_dist_matrix, np.nan)
cax = ax.imshow(dtw_dist_matrix, cmap='Blues', vmin=0, vmax=500)
fig.colorbar(cax, ax=ax)

# Set labels with fontsize 16
ax.set_xlabel("Index", fontsize=16)
ax.set_ylabel("Index", fontsize=16)

# Print information
print(intersection_name)
print(f'{dtw_key}')


# plot optics start end with medoids

In [ ]:
import json

# Save to JSON
file_path = f'{code_root_dir}/src/models/OUTLIER_DETECTION/OPTICS_MODELS/{intersection_name}_optics_optimized_params.json'
print(file_path)
with open(file_path) as f:
    optimization_params = json.load(f)
print(optimization_params)

In [ ]:
optimization_params_silhouette = optimization_params['davies_bouldin']

In [ ]:
from src.models.ensure_distance_metric_params import ensure_distance_metric_params
from src.models.DISTANCE_METRICS_WITH_ADDITIONAL_ARGS import DISTANCE_METRICS_WITH_ADDITIONAL_ARGS
max_eps = optimization_params_silhouette['epsilon']
min_samples = optimization_params_silhouette['min_samples']
metric = optimization_params_silhouette['metric']
cluster_method = optimization_params_silhouette['cluster_method']

xi = optimization_params_silhouette['xi'] if cluster_method == 'xi' else 1

if metric in DISTANCE_METRICS_WITH_ADDITIONAL_ARGS:
    metric_params = ensure_distance_metric_params(X_intersection,[metric])
    optics_model = OPTICS(min_samples=min_samples,
                        cluster_method=cluster_method,
                        metric=metric,
                        max_eps=max_eps,
                        xi=xi,
                        metric_params=metric_params[metric]
                        )
else:
    metric_params = {}
    optics_model = OPTICS(min_samples=min_samples,
                        cluster_method=cluster_method,
                        metric=metric,
                        max_eps=max_eps,
                        xi=xi
                        )

print(metric_params)

optics_model.fit(X_intersection)

# Extract reachability distances and the ordering of points
reachability = optics_model.reachability_
ordering = optics_model.ordering_
clusters = optics_model.labels_

fig, ax2 = plt.subplots(1, 1, figsize=(10, 10))  # Adjust figsize as needed

# Define unique labels (clusters)
unique_labels = np.unique(clusters)
n_clusters = len(unique_labels) - (1 if -1 in clusters else 0)  # Exclude noise

# Generate colors for clusters, excluding noise
colors = plt.cm.viridis(np.linspace(0, 1, n_clusters))

# Create a color mapping for clusters
color_dict = {label: color for label, color in zip(unique_labels[unique_labels != -1], colors)}
color_dict[-1] = 'gray'  # Assign gray color for noise

# Sort reachability distances according to the ordering
sorted_reachability = reachability[ordering]
sorted_labels = clusters[ordering]
sorted_colors = [color_dict[label] for label in sorted_labels]
space = np.arange(len(X_intersection))

# Assign black color for noise if present
if -1 in clusters:
    color_dict[-1] = [0, 0, 0, 1]  # Black color for noise

# Plot each cluster
for label in unique_labels:
    class_member_mask = (clusters == label)
    xy = X_intersection[class_member_mask]
    
    if label == -1:
        # Plot noise
        ax2.plot(xy[:, 0], xy[:, 1], 'o',
                 color='gray',
                 markersize=6,
                 label='Noise',
                 zorder=100)
    else:
        # Plot clustered points
        ax2.plot(xy[:, 0], xy[:, 1], 'o',
                 color=tuple(color_dict[label]),
                 markersize=6,
                 label=f'Cluster {label}',
                 zorder=100)

# Set plot limits based on data
plot_vehicle_tracks_in_notebook(ax2, df_cuid, df_cuid_grouped, color='gray', alpha=0.1)

# ******************
# Calc medoids
# ******************

from src.models.optics.calculate_cluster_medoids import calculate_cluster_medoids
print(len(X_intersection))
medoids = calculate_cluster_medoids(X_intersection, optics_model)
label_flag = False
for medoid in medoids:
    if not label_flag:
        ax2.plot(medoid[0], medoid[1], 'X', color='r', markersize=10, label='Medoid', zorder=10000)
        label_flag = True
    ax2.plot(medoid[0], medoid[1], 'X', color='r', markersize=10, zorder=10000)

# # Set labels and title
# ax2.set_xlabel('X', fontsize=16)
# ax2.set_ylabel('Y', fontsize=16)
# ax2.legend(loc='best', fontsize=10)
# ax2.grid(True, alpha=0.1)
# # create tuple list of vehicle track 43 for k733_2018
# track_no = 102
# track_all = df_cuid_grouped.iloc[track_no]
# track_x = track_all['x']
# track_y = track_all['y']
# ax2.plot(track_x, track_y, color='red', linestyle='-', linewidth=2)


# =========================
# Final Adjustments and Show Plot
# =========================

print(intersection_name)
plt.tight_layout()
plt.show()

# plot optimized dtw clusters

#### see: notebooks/exploratory/plot_optimized_dtw_optics.ipynb

# plot subclusters

In [ ]:
# from src.visualization.plot_within_cluster_optics import plot_within_cluster_optics


# if intersection_name=='k729_2022':
#     intersection_name_k729_2022 = 'k729_2022'
#     params_uid_k729_2022 = 'WDB'
#     cluster_label_k729_2022 = 2
#     eval_metric_k729_2022 = 'silhouette'
#     within_cluster_params_uid_k729_2022 = 'SDB'
#     for cluster_label in range(0,10):
#         plot_within_cluster_optics(intersection_name_k729_2022, params_uid_k729_2022, cluster_label, eval_metric_k729_2022, within_cluster_params_uid_k729_2022)
    
# if intersection_name=='k733_2018':
#     within_cluster_optimization_init_params_id_k733_2018 = 'NCB'
#     intersection_name_k733_2018 = 'k733_2018'
#     intersection_optics_params_uid_k733_2018 = None
#     eval_metric_k733_2018 = 'silhouette'
#     for cluster_label in range(0,7):
#         plot_within_cluster_optics(intersection_name_k733_2018, intersection_optics_params_uid_k733_2018, cluster_label, eval_metric_k733_2018, within_cluster_optimization_init_params_id_k733_2018)

# if intersection_name=='k733_2020':
#     within_cluster_optimization_init_params_id_k733_2020 = 'NCB'
#     intersection_name_k733_2020 = 'k733_2020'
#     intersection_optics_params_uid_k733_2020 = None
#     eval_metric_k733_2020 = 'silhouette'
#     for cluster_label in range(0,5):
#         plot_within_cluster_optics(intersection_name_k733_2020, intersection_optics_params_uid_k733_2020, cluster_label, eval_metric_k733_2020, within_cluster_optimization_init_params_id_k733_2020)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
plot_vehicle_tracks_in_notebook(ax, df_cuid, df_cuid_grouped, color='gray', alpha=0.1)

# plot start and end points w/ scores within cluster

In [ ]:
import json

# Save to JSON
file_path = f'{code_root_dir}/src/models/OUTLIER_DETECTION/OPTICS_MODELS/{intersection_name}_optics_optimized_params.json'
print(file_path)
with open(file_path) as f:
    optimization_params = json.load(f)
print(optimization_params)
optimization_params_silhouette = optimization_params['calinski_harabasz']
from src.models.ensure_distance_metric_params import ensure_distance_metric_params
from src.models.DISTANCE_METRICS_WITH_ADDITIONAL_ARGS import DISTANCE_METRICS_WITH_ADDITIONAL_ARGS
max_eps = optimization_params_silhouette['epsilon']
min_samples = optimization_params_silhouette['min_samples']
metric = optimization_params_silhouette['metric']
cluster_method = optimization_params_silhouette['cluster_method']

xi = optimization_params_silhouette['xi'] if cluster_method == 'xi' else 1

if metric in DISTANCE_METRICS_WITH_ADDITIONAL_ARGS:
    metric_params = ensure_distance_metric_params(X_intersection,[metric])
    optics_model = OPTICS(min_samples=min_samples,
                        cluster_method=cluster_method,
                        metric=metric,
                        max_eps=max_eps,
                        xi=xi,
                        metric_params=metric_params[metric]
                        )
else:
    metric_params = {}
    optics_model = OPTICS(min_samples=min_samples,
                        cluster_method=cluster_method,
                        metric=metric,
                        max_eps=max_eps,
                        xi=xi
                        )

print(metric_params)

optics_model.fit(X_intersection)

clusters = optics_model.labels_

fig, ax2 = plt.subplots(1, 1, figsize=(10, 10))  # Adjust figsize as needed

# Define unique labels (clusters)
unique_labels = np.unique(clusters)
n_clusters = len(unique_labels) - (1 if -1 in clusters else 0)  # Exclude noise

# Generate colors for clusters, excluding noise
colors = plt.cm.viridis(np.linspace(0, 1, n_clusters))

# Create a color mapping for clusters
color_dict = {label: color for label, color in zip(unique_labels[unique_labels != -1], colors)}
color_dict[-1] = 'gray'  # Assign gray color for noise

# Sort reachability distances according to the ordering
sorted_reachability = reachability[ordering]
sorted_labels = clusters[ordering]
sorted_colors = [color_dict[label] for label in sorted_labels]
space = np.arange(len(X_intersection))

# Assign black color for noise if present
if -1 in clusters:
    color_dict[-1] = [0, 0, 0, 1]  # Black color for noise

alpha_val = .2

# Plot each cluster
for label in unique_labels:
    class_member_mask = (clusters == label)
    xy = X_intersection[class_member_mask]
    
    if label == -1:
        # Plot noise
        ax2.plot(xy[:, 0], xy[:, 1], 'o',
                 color='gray',
                 markersize=6,
                 label='Noise',
                 zorder=100,
                 alpha=alpha_val)
    else:
        # Plot clustered points
        ax2.plot(xy[:, 0], xy[:, 1], 'o',
                 color=tuple(color_dict[label]),
                 markersize=6,
                 label=f'Cluster {label}',
                 zorder=100,
                 alpha=alpha_val)

# Set plot limits based on data
plot_vehicle_tracks_in_notebook(ax2, df_cuid, df_cuid_grouped, color='gray', alpha=0.1)

# ******************
# Calc medoids
# ******************

alpha_medoids = .5

from src.models.optics.calculate_cluster_medoids import calculate_cluster_medoids
print(len(X_intersection))
medoids = calculate_cluster_medoids(X_intersection, optics_model)
label_flag = False
for medoid in medoids:
    if not label_flag:
        ax2.plot(medoid[0], medoid[1], 'X', color='r', markersize=10, label='Medoid', zorder=10000, alpha=alpha_medoids)
        label_flag = True
    ax2.plot(medoid[0], medoid[1], 'X', color='r', markersize=10, zorder=10000, alpha=alpha_medoids)

# Set labels and title
ax2.set_xlabel('X', fontsize=16)
ax2.set_ylabel('Y', fontsize=16)
ax2.grid(True, alpha=0.1)
# create tuple list of vehicle track 43 for k733_2018
track_no = 102
track_all = df_cuid_grouped.iloc[track_no]
track_x = track_all['x']
track_y = track_all['y']
x_start = track_all['x'][0]
y_start = track_all['y'][0]
x_end = track_all['x'][-1]
y_end = track_all['y'][-1]
ax2.plot(track_x, track_y, color='black', linestyle='-', linewidth=4, alpha=1, zorder=100000)
ax2.plot(x_start, y_start, 'o', color='green', markersize=12, label='Start', zorder=100000)
ax2.plot(x_end, y_end, 'o', color='red', markersize=12, label='End', zorder=100000)
ax2.legend(loc='best', fontsize=10)
# add text below plot
start_score = 0.44966442953020136
end_score = 0.2818791946308724

fig.text(0.32, -0.02, f'Start Point Outlier Probability Score:      {start_score:.4f}', ha='center', fontsize=16)
fig.text(0.32, -0.05, f'End Point Outlier Probability Score:        {end_score:.4f}', ha='center', fontsize=16)


# =========================
# Final Adjustments and Show Plot
# =========================

print(intersection_name)
plt.tight_layout()
plt.show()

# plot path w/ path score w/ subclusters

In [ ]:
from src.features.get_x_y_tuple_list import get_x_y_tuple_list
from src.visualization.render_vehicle_track_cluster_in_notebook import render_vehicle_track_cluster_in_notebook
from src.models.optics.get_clusters_from_optics_labels import get_clusters_from_optics_labels
from src.visualization.plot_vehicle_tracks_in_notebook import plot_vehicle_tracks_in_notebook


def plot_within_cluster_optics(intersection_name, params_uid, cluster_label, eval_metric, within_cluster_params_uid=None):

    print(f'------------------------within CLUSTER #{cluster_label} for INTERSECTION {intersection_name}------------------------')

    # get data
    data_path = f'{code_root_dir}/data/processed/{intersection_name}_cuid.csv'
    df_cuid = pd.read_csv(data_path)
    df_cuid_grouped_path = data_path.replace('.csv', '_grouped.csv')
    df_cuid_grouped = pd.read_csv(df_cuid_grouped_path)
    df_cuid_grouped['x'] = df_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
    df_cuid_grouped['y'] = df_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))
    list_x_y_tuples = get_x_y_tuple_list(df_cuid_grouped, ['x','y'])

    # ----------------- UNDERLYING OPTICS -----------------
    def get_optics_params(optics_params, metric):
        optics_params = optics_params[metric]
        dtw_dist_matrix_key = optics_params['dtw_key']
        max_eps = optics_params['epsilon']
        min_samples = optics_params['min_samples']
        cluster_method = optics_params['cluster_method']
        xi = optics_params['xi']
        kwargs = {
            'max_eps': max_eps,
            'min_samples': min_samples,
            'cluster_method': cluster_method,
            'xi': xi
        }
        filtered_kwargs = {k: v for k, v in kwargs.items() if v is not None}
        return dtw_dist_matrix_key, filtered_kwargs
    
    # check for params uid
    if params_uid is not None:
        opt_params_path = f'{code_root_dir}/data/processed/{intersection_name}_optics_vehicle_paths_optimized_params_{params_uid}.json'
    else:
        opt_params_path = f'{code_root_dir}/data/processed/{intersection_name}_optics_vehicle_paths_optimized_params.json'
    with open(opt_params_path, 'r') as f:
        optics_params = json.load(f)

    # import optimized parameters
    dtw_matrix_key, kwargs_acc_score = get_optics_params(optics_params, eval_metric)

    # import dtw distance matrix
    dtw_distance_matrix_path = f'{code_root_dir}/data/processed/{intersection_name}_diff_itakura_slope_dtw_matrices.json'
    with open(dtw_distance_matrix_path) as f:
        dtw_distance_matrices_dict = json.load(f)

    # get the distance matrix
    dtw_distance_matrix = dtw_distance_matrices_dict[dtw_matrix_key]

    # create optics model
    optics_model = OPTICS(metric='precomputed', **kwargs_acc_score).fit(dtw_distance_matrix)

    # get labels for big models
    labels = get_clusters_from_optics_labels(optics_model.labels_)

    



    # ----------------- WITHIN CLUSTER OPTICS -----------------
    # path to optimized within cluster optics params
    path_optimized_in_cluster_optics_params = f'{code_root_dir}/data/processed/within_cluster_clustering_optimization/{intersection_name}_optics_optimized_vehicle_paths_{params_uid}_{eval_metric}_within_cluster_{cluster_label}_optimized_params_{within_cluster_params_uid}.json'

    # load optimized within cluster optics params
    try:
        with open(path_optimized_in_cluster_optics_params, 'r') as f:
            within_cluster_optics_params = json.load(f)
    except FileNotFoundError as e:
        print(f'File not found: {path_optimized_in_cluster_optics_params}')
        _, ax = plt.subplots(1, 1, figsize=(10,10))
        # filter labels to include whole cluster
        render_vehicle_track_cluster_in_notebook(ax, df_cuid, df_cuid_grouped, {'0': labels[cluster_label]})
        plot_vehicle_tracks_in_notebook(ax, df_cuid, df_cuid_grouped, color='gray', alpha=0.5, title=f'Intersection: {intersection_name} \n No within cluster optimization found for cluster #{cluster_label}')
        return
    
    # extract within cluster optics params according to eval metric
    within_cluster_optics_params = within_cluster_optics_params[eval_metric]

    # extract within cluster optics params
    within_cluster_max_eps = within_cluster_optics_params['epsilon']
    within_cluster_min_samples = within_cluster_optics_params['min_samples']
    within_cluster_cluster_method = within_cluster_optics_params['cluster_method']
    within_cluster_xi = within_cluster_optics_params['xi'] if within_cluster_optics_params['xi'] is not None else 0.05
    within_cluster_dtw_key = within_cluster_optics_params['dtw_key']

    # create filtered data using certain cluster
    df_grouped_within_cluster_filtered = df_cuid_grouped.loc[optics_model.labels_ == cluster_label] 

    # extract optimized within cluster dtw distance matrix
    within_cluster_dtw_distance_matrix = dtw_distance_matrices_dict[within_cluster_dtw_key]

    # create within cluster dtw distance matrices by filtering only for the cluster
    within_cluster_dtw_distance_matrix = np.array(within_cluster_dtw_distance_matrix)
    filtered_rows = within_cluster_dtw_distance_matrix[optics_model.labels_ == cluster_label]
    within_cluster_dtw_distance_matrix = filtered_rows[:, optics_model.labels_ == cluster_label]

    # fit new within cluster optics model
    optics_within_cluster = OPTICS(metric='precomputed', 
                                   max_eps=within_cluster_max_eps, 
                                   min_samples=within_cluster_min_samples, 
                                   cluster_method=within_cluster_cluster_method, 
                                   xi=within_cluster_xi
                                   ).fit(within_cluster_dtw_distance_matrix)

    # get labels for within cluster model
    labels_within_cluster = get_clusters_from_optics_labels(optics_within_cluster.labels_)
    # path_score = 0.998894239896124
    path_score = 0.016350845506988853


    # plot within cluster clusters
    fig, axs = plt.subplots(1, 1, figsize=(10,10))
    fig.text(0.365, 0.0, f'Path Outlier Probability Score:        {path_score:.4f}', ha='center', fontsize=16)
    # track_all = [(1.4866181033949353, -11.157597292613953), (1.6007128092120777, -11.366445569474749), (1.71480751502922, -11.575293846335542), (1.8289022208463623, -11.784142123196338), (1.9429969266635045, -11.992990400057133), (2.057091632480647, -12.201838676917927), (2.1711863382977894, -12.410686953778722), (2.2852810441149316, -12.619535230639517), (2.399375749932074, -12.82838350750031), (2.5134704557492165, -13.037231784361106), (2.627565161566359, -13.246080061221901), (2.7416598673835013, -13.454928338082695), (2.8557545732006435, -13.66377661494349), (2.9698492790177857, -13.872624891804286), (3.083943984834928, -14.08147316866508), (3.19803869065207, -14.290321445525874), (3.312133396469213, -14.499169722386668), (3.4262281022863554, -14.708017999247463), (3.5403228081034976, -14.916866276108259), (3.65441751392064, -15.125714552969054), (3.7685122197377825, -15.33456282982985), (3.8826069255549243, -15.543411106690641), (3.996701631372067, -15.752259383551436), (4.110796337189209, -15.961107660412232), (4.224891043006352, -16.169955937273027), (4.338985748823494, -16.378804214133822), (4.453080454640636, -16.587652490994614), (4.567175160457778, -16.79650076785541), (4.681269866274921, -17.005349044716205), (4.795364572092064, -17.214197321577), (4.909459277909205, -17.423045598437795), (5.023553983726348, -17.631893875298587), (5.13764868954349, -17.840742152159386), (5.2517433953606325, -18.049590429020178), (5.365838101177776, -18.258438705880973), (5.479932806994917, -18.46728698274177), (5.59402751281206, -18.676135259602564), (5.708122218629201, -18.88498353646336), (5.822216924446344, -19.09383181332415), (5.936311630263487, -19.30268009018495), (6.05040633608063, -19.51152836704574), (6.164501041897771, -19.720376643906537), (6.278595747714913, -19.929224920767332), (6.392690453532056, -20.138073197628124), (6.5067851593491985, -20.346921474488923), (6.620879865166341, -20.555769751349715), (6.734974570983483, -20.76461802821051), (6.849069276800625, -20.973466305071305), (6.963163982617768, -21.1823145819321), (7.0772586884349105, -21.391162858792896), (7.191353394252053, -21.600011135653688), (7.305448100069195, -21.808859412514487), (7.419542805886337, -22.01770768937528), (7.53363751170348, -22.226555966236074), (7.6477322175206215, -22.43540424309687), (7.761826923337765, -22.64425251995766), (7.875921629154907, -22.85310079681846), (7.990016334972049, -23.06194907367925), (8.104111040789192, -23.270797350540047), (8.218205746606333, -23.479645627400842)]
    # track_x = [x[0] for x in track_all]
    # track_y = [y[1] for y in track_all]
    # axs.plot(track_x, track_y, color='red', linestyle='-', linewidth=4, alpha=1, zorder=100000)    
    # # track_no = 102
    track_all = df_cuid_grouped.iloc[track_no]
    axs.plot(track_all['x'], track_all['y'], color='red', linestyle='-', linewidth=4, alpha=1, zorder=100000)

    render_vehicle_track_cluster_in_notebook(axs, df_cuid, df_grouped_within_cluster_filtered, labels_within_cluster)
    plot_vehicle_tracks_in_notebook(axs, df_cuid, df_cuid_grouped, '', color='gray', alpha=0.5)
    # delete legend from axs
    axs.get_legend().remove()

In [ ]:
intersection_name_k729_2022 = 'k729_2022'
params_uid_k729_2022 = 'WDB'
eval_metric_k729_2022 = 'silhouette'
within_cluster_params_uid_k729_2022 = 'SDB'

In [ ]:
# for cluster_label in range(0,10):
#     plot_within_cluster_optics(intersection_name_k729_2022, params_uid_k729_2022, cluster_label, eval_metric_k729_2022, within_cluster_params_uid_k729_2022)

cluster_label = 0
plot_within_cluster_optics(intersection_name_k729_2022, params_uid_k729_2022, cluster_label, eval_metric_k729_2022, within_cluster_params_uid_k729_2022)
